In [1]:
google_colab = False

# Google Colab用事前準備

In [2]:
# install
try:
    import binarybrain as bb
except:
    if google_colab and False:
        !pip install pybind11
        %cd /content
        !nvcc -V
        !sudo rm -fr BinaryBrain
        !rm -fr BinaryBrain
        !git clone --recursive -b ver3_develop https://github.com/ryuz/BinaryBrain.git
        %cd /content/BinaryBrain/python
        !sh copy_src.sh
        !python3 setup.py build
        !python3 setup.py develop

        import binarybrain as bb

In [3]:
# mount
if google_colab:
  from google.colab import drive 
  drive.mount('/content/drive')
  %cd /content/drive/My Drive/git-work/BinaryBrain_ver3_develop/tests/python

In [17]:
import binarybrain as bb

import importlib
importlib.reload(bb)

prop = bb.get_device_properties()
print(prop)


name                     : GeForce GTX 1660 SUPER
totalGlobalMem           : 6442450944
sharedMemPerBlock        : 49152
regsPerBlock             : 65536
warpSize                 : 32
memPitch                 : 2147483647
maxThreadsPerBlock       : 1024
maxThreadsDim[0]         : 1024
maxThreadsDim[1]         : 1024
maxThreadsDim[2]         : 64
maxGridSize[0]           : 2147483647
maxGridSize[1]           : 65535
maxGridSize[2]           : 65535
clockRate                : 1800000
totalConstMem            : 65536
major                    : 7
minor                    : 5
textureAlignment         : 512
deviceOverlap            : 1
multiProcessorCount      : 22
kernelExecTimeoutEnabled : 1
integrated               : 0
canMapHostMemory         : 1
computeMode              : 0




# メインコード

In [5]:
import binarybrain as bb
import numpy as np
import os
import sys
from tqdm import tqdm
from collections import OrderedDict

In [6]:
# load MNIST data
td = bb.load_mnist()

# ネットワーク構築

In [7]:
data_path = 'MnistMobileNetDistillation'
os.makedirs(data_path, exist_ok=True)

ref_affine_list = []
ref_norm_list   = []
target_lut_list = []

def clear_list():
    ref_affine_list.clear()
    ref_norm_list.clear()
    target_lut_list.clear()

def save_model_list(model_list, name):
    for i, model in enumerate(model_list):
        model.save_json(os.path.join(data_path, '%s_%d.json' % (name, i)))

def load_model_list(model_list, name):
    for i, model in enumerate(model_list):
        model.load_json(os.path.join(data_path, '%s_%d.json' % (name, i)))

def save_all_model():
    save_model_list(ref_affine_list, 'ref_affine')
    save_model_list(ref_norm_list,   'ref_norm_list')
    save_model_list(target_lut_list, 'target_lut')
    
def load_all_model():
    load_model_list(ref_affine_list, 'ref_affine')
    load_model_list(ref_norm_list,   'ref_norm_list')
    load_model_list(target_lut_list, 'target_lut')

def make_common_layer(model):
    layer = {}
    layer['type']   = 'common'
    layer['select'] = 'common'
    layer['common'] = model
    return layer    

def make_cnv_layer(ch_size, w=3, h=3, connection='random'):
    # setup infomation
    layer = {}
    layer['type']            = 'convolution'
    layer['select']          = 'ref'
    layer['connection']      = connection
    layer['target_lut_size'] = 2
    
    # create model
    if connection == 'depthwise':
        layer['ref_affine']  = bb.DepthwiseDenseAffine.create([ch_size])
        layer['target_lut0'] = bb.SparseLut6.create([1, 1, ch_size], connection='depthwise')
        layer['target_lut1'] = bb.SparseLut6.create([6, 1, ch_size], connection='depthwise')
    else:
        layer['ref_affine']  = bb.DenseAffine.create([ch_size])
        layer['target_lut0'] = bb.SparseLut6.create([1, 1, ch_size])
        layer['target_lut1'] = bb.SparseLut6.create([1, 1, ch_size*6])
    layer['ref_norm']        = bb.BatchNormalization.create()
    layer['ref_act']         = bb.ReLU.create()
    
    # save
    ref_affine_list.append(layer['ref_affine'])
    ref_norm_list.append(layer['ref_norm'])
    target_lut_list.append(layer['target_lut1'])
    target_lut_list.append(layer['target_lut0'])
    
    # make network
    ref_subnet =  bb.Sequential.create()
    ref_subnet.add(layer['ref_affine'])
    ref_subnet.add(layer['ref_norm'])
    ref_subnet.add(layer['ref_act'])
    layer['ref'] = bb.LoweringConvolution.create(ref_subnet, w, h)
    
    target_subnet = bb.Sequential.create()
    target_subnet.add(layer['target_lut1'])
    target_subnet.add(layer['target_lut0'])
    layer['target'] = bb.LoweringConvolution.create(target_subnet, w, h)
    
    return layer

def build_net(layer_list):
    net = bb.Sequential.create()
    for layer in layer_list:
        net.add(layer[layer['select']])
    return net

In [8]:
# build network
modulation_size = 8
layer_rel2bin = bb.RealToBinary.create(modulation_size, framewise=True)
layer_bin2rel = bb.BinaryToReal.create(modulation_size)

clear_list()
layer_list = []
layer_list.append(make_common_layer(layer_rel2bin))
layer_list.append(make_cnv_layer(36, 3, 3, 'random'))      # 26x26
layer_list.append(make_cnv_layer(36, 3, 3, 'depthwise'))   # 24x24
layer_list.append(make_cnv_layer(36, 1, 1, 'pointwise'))
layer_list.append(make_common_layer(bb.MaxPooling.create(2, 2)))  # 12x12
layer_list.append(make_cnv_layer(36, 3, 3, 'depthwise'))   # 10x10
layer_list.append(make_cnv_layer(36, 1, 1, 'pointwise'))
layer_list.append(make_cnv_layer(36, 3, 3, 'depthwise'))   # 8x8
layer_list.append(make_cnv_layer(36, 1, 1, 'pointwise'))
layer_list.append(make_common_layer(bb.MaxPooling.create(2, 2)))  # 4x4
layer_list.append(make_cnv_layer(36, 3, 3, 'depthwise'))   # 2x2
layer_list.append(make_cnv_layer(36, 1, 1, 'pointwise'))
layer_list.append(make_cnv_layer(36, 2, 2, 'depthwise'))   # 1x1
layer_list.append(make_cnv_layer(10, 1, 1, 'pointwise'))
layer_list.append(make_common_layer(layer_bin2rel))

In [9]:
main_net = build_net(layer_list)
main_net.set_input_shape(td['x_shape'])
main_net.send_command('binary true')

In [10]:
#load_all_model()

In [11]:
print(main_net.get_info())

----------------------------------------------------------------------
[Sequential] 
  --------------------------------------------------------------------
  [RealToBinary] 
   input  shape : {28, 28, 1}   output shape : {28, 28, 1}
  --------------------------------------------------------------------
  [LoweringConvolution] 
   filter size : (3, 3)
   input  shape : {28, 28, 1}   output shape : {26, 26, 36}
    ------------------------------------------------------------------
    [ConvolutionIm2Col] 
     input  shape : {28, 28, 1}     output shape : {3, 3, 1}
    ------------------------------------------------------------------
    [Sequential] 
      ----------------------------------------------------------------
      [DenseAffine] 
       input  shape : {3, 3, 1}       output shape : {36}
      ----------------------------------------------------------------
      [BatchNormalization] 
       input  shape : {36}       output shape : {36}
      ---------------------------------

In [12]:
# train
loss      = bb.LossSoftmaxCrossEntropy.create()
metrics   = bb.MetricsCategoricalAccuracy.create()
optimizer = bb.OptimizerAdam.create()
optimizer.set_variables(main_net.get_parameters(), main_net.get_gradients())

runner = bb.Runner(main_net, "mnist-mobile-net", loss, metrics, optimizer)
runner.fitting(td, epoch_size=32, mini_batch_size=32, file_write=False, file_read=False)

  0%|                                                                                         | 0/1875 [00:00<?, ?it/s]

epoch=1 test_accuracy=0.894100 test_loss=1.578065 train_accuracy=0.886267 train_loss=1.582979


  0%|                                                                                         | 0/1875 [00:00<?, ?it/s]

epoch=2 test_accuracy=0.943500 test_loss=1.530398 train_accuracy=0.937533 train_loss=1.536215


  0%|                                                                                         | 0/1875 [00:00<?, ?it/s]

epoch=3 test_accuracy=0.954200 test_loss=1.523026 train_accuracy=0.950217 train_loss=1.528064


KeyboardInterrupt: 

In [13]:
save_model_list(ref_affine_list, 'ref_affine')
save_model_list(ref_norm_list,   'ref_norm_list')
#save_all_model()

In [ ]:
## switch_model
#for layer in layer_list:
#    if layer['type'] == 'convolution':
#        layer['select'] = 'target'

#main_net = build_net(layer_list)
#main_net.set_input_shape(td['x_shape'])
#main_net.send_command('binary true')

#print(main_net.get_info())

In [ ]:
#save_all_model()

In [14]:
def distillation_layer(layer):
    if layer['type'] != 'convolution':
        return False
    
    ref_affine  = layer['ref_affine']
    target_lut0 = layer['target_lut0']
    target_lut1 = layer['target_lut1']
    
    if False: # layer['connection'] == 'pointwise':
        tensorW = layer['ref_affine'].W()
        W = np.array(tensorW.get_data()).reshape(tensorW.get_shape()[::-1])
        idx = np.argsort(-np.abs(W), axis=1)
        
        # Weight順で接続
        lut0_input_shape  = target_lut0.get_input_shape()
        lut0_output_shape = target_lut0.get_output_shape()
        for i in range(lut0_output_shape[2]):
            for j in range(6):
                target_lut0.set_node_connection_index(i, j, i*6+j)
        
        lut1_input_node_size  = target_lut1.get_input_node_size()
        lut1_output_node_size = target_lut1.get_output_node_size()
        for i in range(lut1_output_node_size//6):
            for j in range(6):
                for k in range(6):
                    target_lut1.set_node_connection_index(i*6+j, k, idx[i][j*6+k])
    
    layer['select'] = 'target'
    return True

In [15]:
print(distillation_layer(layer_list[1]))

main_net = build_net(layer_list)
main_net.set_input_shape(td['x_shape'])
main_net.send_command('binary true')

print(main_net.get_info())

True
----------------------------------------------------------------------
[Sequential] 
  --------------------------------------------------------------------
  [RealToBinary] 
   input  shape : {28, 28, 1}   output shape : {28, 28, 1}
  --------------------------------------------------------------------
  [LoweringConvolution] 
   filter size : (3, 3)
   input  shape : {28, 28, 1}   output shape : {26, 26, 36}
    ------------------------------------------------------------------
    [ConvolutionIm2Col] 
     input  shape : {28, 28, 1}     output shape : {3, 3, 1}
    ------------------------------------------------------------------
    [Sequential] 
      ----------------------------------------------------------------
      [SparseLut6] 
       input  shape : {3, 3, 1}       output shape : {1, 1, 216}
       binary : 1       batch_norm : 1
      ----------------------------------------------------------------
      [SparseLut6] 
       input  shape : {1, 1, 216}       output sha

In [16]:
# train
if True:
    for i in range(len(layer_list)):
        if distillation_layer(layer_list[i]):
            main_net = build_net(layer_list)
            main_net.set_input_shape(td['x_shape'])
            main_net.send_command('binary true')

            print(main_net.get_info())

            loss      = bb.LossSoftmaxCrossEntropy.create()
            metrics   = bb.MetricsCategoricalAccuracy.create()
            optimizer = bb.OptimizerAdam.create()
            optimizer.set_variables(main_net.get_parameters(), main_net.get_gradients())

            runner = bb.Runner(main_net, "mnist-mobile-net", loss, metrics, optimizer)
            runner.fitting(td, epoch_size=1, mini_batch_size=32, file_write=False, file_read=False)

  0%|                                                                                         | 0/1875 [00:00<?, ?it/s]

----------------------------------------------------------------------
[Sequential] 
  --------------------------------------------------------------------
  [RealToBinary] 
   input  shape : {28, 28, 1}   output shape : {28, 28, 1}
  --------------------------------------------------------------------
  [LoweringConvolution] 
   filter size : (3, 3)
   input  shape : {28, 28, 1}   output shape : {26, 26, 36}
    ------------------------------------------------------------------
    [ConvolutionIm2Col] 
     input  shape : {28, 28, 1}     output shape : {3, 3, 1}
    ------------------------------------------------------------------
    [Sequential] 
      ----------------------------------------------------------------
      [SparseLut6] 
       input  shape : {3, 3, 1}       output shape : {1, 1, 216}
       binary : 1       batch_norm : 1
      ----------------------------------------------------------------
      [SparseLut6] 
       input  shape : {1, 1, 216}       output shape : 

epoch=1 test_accuracy=0.933000 test_loss=1.536400 train_accuracy=0.925500 train_loss=1.543399
----------------------------------------------------------------------
[Sequential] 
  --------------------------------------------------------------------
  [RealToBinary] 
   input  shape : {28, 28, 1}   output shape : {28, 28, 1}
  --------------------------------------------------------------------
  [LoweringConvolution] 
   filter size : (3, 3)
   input  shape : {28, 28, 1}   output shape : {26, 26, 36}
    ------------------------------------------------------------------
    [ConvolutionIm2Col] 
     input  shape : {28, 28, 1}     output shape : {3, 3, 1}
    ------------------------------------------------------------------
    [Sequential] 
      ----------------------------------------------------------------
      [SparseLut6] 
       input  shape : {3, 3, 1}       output shape : {1, 1, 216}
       binary : 1       batch_norm : 1
      ----------------------------------------------

epoch=1 test_accuracy=0.947000 test_loss=1.527648 train_accuracy=0.944117 train_loss=1.533427
----------------------------------------------------------------------
[Sequential] 
  --------------------------------------------------------------------
  [RealToBinary] 
   input  shape : {28, 28, 1}   output shape : {28, 28, 1}
  --------------------------------------------------------------------
  [LoweringConvolution] 
   filter size : (3, 3)
   input  shape : {28, 28, 1}   output shape : {26, 26, 36}
    ------------------------------------------------------------------
    [ConvolutionIm2Col] 
     input  shape : {28, 28, 1}     output shape : {3, 3, 1}
    ------------------------------------------------------------------
    [Sequential] 
      ----------------------------------------------------------------
      [SparseLut6] 
       input  shape : {3, 3, 1}       output shape : {1, 1, 216}
       binary : 1       batch_norm : 1
      ----------------------------------------------

  0%|                                                                                         | 0/1875 [00:00<?, ?it/s]

KeyboardInterrupt: 